In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as trans
import torchvision as tv

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.linalg as LN

import os,time,random
os.sys.path.insert(0,'../')
import utils
import models
import attacks

In [92]:
batch_size=200
model_path = '../pretrains/store/resnet8-cifar10-87.25.pkl'
model_arch = 'resnet8'
B    = 5.0/255
step = 1
eps  = 5.0/255

train_set = dsets.CIFAR10(root='/home/leiwu/data/cifar10/',train=True,transform=trans.ToTensor(),download=True)
test_set  = dsets.CIFAR10(root='/home/leiwu/data/cifar10/',train=False,transform=trans.ToTensor(),download=False)
trX,trY   = torch.from_numpy(train_set.train_data.copy()).float()/255.0,torch.LongTensor(train_set.train_labels)
teX, teY  = torch.from_numpy(test_set.test_data.copy()).float()/255.0,   torch.LongTensor(test_set.test_labels)

train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=False)
test_loader  = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size,shuffle=False)

model       = models.__dict__[model_arch]().cuda()
model_state = torch.load(model_path)
model.load_state_dict(model_state)
ct    = nn.CrossEntropyLoss().cuda()

def show(img,nrow=8):
    if isinstance(img,np.ndarray):
        img = torch.from_numpy(img)
    img = tv.utils.make_grid(img,nrow=nrow)
    npimg = img.numpy()
    npimg = np.transpose(npimg,(1,2,0))
    plt.imshow(npimg,interpolation='nearest')

def batch_attack(model,ct,lr,eps,niter=1):
    adv_img = torch.zeros(10000,3,32,32)

    for i,(x,y) in enumerate(test_loader):
        if niter==1:
            lr = eps
        adv_x = attacks.gsm(model,ct,x.cuda(),y.cuda(),lr=lr,eps=eps,niter=niter)
        adv_img[i*batch_size:(i+1)*batch_size] = adv_x
    return adv_img

Files already downloaded and verified
resnet: depth: 8, # of blocks at each stage: 1


In [102]:
adv_x = batch_attack(model,ct,lr=B,eps=eps,niter=step)
adv_pred = utils.predictor(model,adv_x.cuda())
clean_pred = utils.predictor(model,teX.cuda())
correct  = adv_pred.eq(clean_pred).squeeze()
wrong    = torch.ones(10000).byte()-correct
wrong_idx = wrong.nonzero()

In [97]:
wrong_idx = wrong.nonzero()
adv_pred_wrong = adv_pred.index_select()

In [101]:
wrong.squeeze().nonzero()


    0
    2
    4
  ⋮   
 9997
 9998
 9999
[torch.LongTensor of size 8467x1]